Cox's PH on original data data, plot survival curves, predict using median survival time

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob, inspect, sys

import warnings
from sklearn.exceptions import ConvergenceWarning

from sklearn.preprocessing import MinMaxScaler
from lifelines import CoxPHFitter

In [ ]:
df = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Merged_data/Survival_df.csv'),
                  index_col=0)

In [ ]:
df_scale=df.copy()

In [ ]:
df_scale[[ 'diff_two_peaks_ratio',
       'amp_ratio', 'pos_ratio', 'energy_ratio', 'NLE_ratio_51_17',
       'NLE_ratio_85_17', 'NLE_ratio_51_119', 'NLO_avg', 'Avg_RP',
       'ACEE_30_Avg_58', 'ACEE_500_Avg_58', 'ACPD_Avg_58']] = MinMaxScaler().fit_transform(df_scale[[ 'diff_two_peaks_ratio',
       'amp_ratio', 'pos_ratio', 'energy_ratio', 'NLE_ratio_51_17',
       'NLE_ratio_85_17', 'NLE_ratio_51_119', 'NLO_avg', 'Avg_RP',
       'ACEE_30_Avg_58', 'ACEE_500_Avg_58', 'ACPD_Avg_58']])

# Cox Proportional Hazard Model Using Lifelines

In [ ]:
#drop to avoid convergence warning
#df_scale.drop(['pos_ratio', 'NLE_ratio_85_17','NLE_ratio_51_17'], axis=1, inplace=True) 

In [ ]:

cph = CoxPHFitter()
surv_func=cph.fit(df_scale, 'F_Time', event_col='Observed')
cph.print_summary()

In [ ]:
cph.plot()

# predicting survival function for df

In [ ]:
fig= plt.figure(figsize=(12,8))

surv_function=cph.predict_survival_function(df_scale)
ID=df_scale.index
plt.rc('font', size=11) 
plt.xlabel("F_Time")
plt.ylabel("Survival Function")
plt.plot(surv_function)
plt.legend(ID) 
plt.show()


In [ ]:
cph.predict_survival_function(df_scale)

In [ ]:
f_df_scale=df_scale[~df_scale.Observed]
t_df_scale=df_scale[df_scale.Observed]

In [ ]:
#Failed
cph.predict_survival_function(t_df_scale)

In [ ]:
fig= plt.figure(figsize=(12,8))

ax=cph.predict_survival_function(t_df_scale)
ID=t_df_scale.index
plt.rc('font', size=11) 
plt.plot(ax)
plt.xlabel("F_Time")
plt.ylabel("Survival Function")
plt.legend(ID) 
plt.show()


In [ ]:
#Not failed
cph.predict_survival_function(f_df_scale)

In [ ]:

fig= plt.figure(figsize=(12,8))

ax=cph.predict_survival_function(f_df_scale)
ID=f_df_scale.index
plt.rc('font', size=11) 
plt.plot(ax)
plt.xlabel("F_Time")
plt.ylabel("Survival Function")
plt.legend(ID) 
plt.show()
